# Βήμα 1: FETCH WIKIPEDIA

## Κελί 1: Εισαγωγή Βιβλιοθηκών
Εισαγωγή των βιβλιοθηκών που θα χρησιμοποιηθούν:

In [5]:
import requests
from bs4 import BeautifulSoup
import json

## Κελί 2: Υλοποίηση Συνάρτησης
Γράφεις τη συνάρτηση fetch_wikipedia_articles:

In [6]:
def fetch_wikipedia_articles(keyword, max_articles=5):
    """
    Συλλέγει άρθρα από το Wikipedia για το συγκεκριμένο keyword, αποφεύγοντας τα διπλότυπα.
    """
    base_url = "https://en.wikipedia.org/wiki/"
    search_url = f"https://en.wikipedia.org/w/index.php?search={keyword.replace(' ', '+')}"
    response = requests.get(search_url)

    if response.status_code != 200:
        print(f"Failed to fetch articles for keyword: {keyword}")
        return []

    soup = BeautifulSoup(response.content, 'html.parser')
    articles = []
    seen_titles = set()  # Χρησιμοποιούμε ένα σύνολο για να παρακολουθούμε μοναδικούς τίτλους
    links = soup.find_all('a', href=True)

    for link in links:
        if len(articles) >= max_articles:
            break
        href = link['href']
        if href.startswith("/wiki/") and not any(prefix in href for prefix in [":", "#"]):
            full_url = f"https://en.wikipedia.org{href}"
            article_response = requests.get(full_url)
            if article_response.status_code == 200:
                article_soup = BeautifulSoup(article_response.content, 'html.parser')
                title = article_soup.find('h1', {'id': 'firstHeading'}).text if article_soup.find('h1', {'id': 'firstHeading'}) else "No Title"
                content = '\n'.join([p.text for p in article_soup.find_all('p')])

                # Έλεγχος για διπλότυπους τίτλους
                if title not in seen_titles:
                    articles.append({"title": title, "content": content})
                    seen_titles.add(title)  # Προσθήκη του τίτλου στο σύνολο

    return articles


## Κελί 3: Κλήση της Συνάρτησης και Αποθήκευση
Εδώ καλείς τη συνάρτηση και αποθηκεύεις τα δεδομένα:

In [7]:
# Εισαγωγή από τον χρήστη
topic = input("Enter the topic: ")
num_articles = int(input("Enter the number of articles: "))

# Κλήση της συνάρτησης
articles = fetch_wikipedia_articles(topic, num_articles)

# Αποθήκευση άρθρων σε JSON αρχείο
with open('articles.json', 'w', encoding='utf-8') as f:
    json.dump(articles, f, indent=4, ensure_ascii=False)

print(f"Saved {len(articles)} articles to 'articles.json'.")


Enter the topic:  maths
Enter the number of articles:  100


Saved 100 articles to 'articles.json'.
